In [4]:
# Load necessary libraries
install.packages("bigrquery")
install.packages("dplyr")
library(bigrquery)
library(dplyr)
# Set your Google Cloud project ID
project_id <- "yhcr-prd-phm-bia-core"

# Set the dataset you want to search
dataset_id <- "CB_FDM_PrimaryCare"

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [7]:
# Load necessary libraries

# Set your Google Cloud project ID
#project_id <- "your-google-cloud-project-id"

# Set the dataset you want to search
#dataset_id <- "your-dataset-id"

# The phrase or word you want to search for
search_phrase <- "Asthma"

# Get a list of all tables in the dataset
tables <- bq_dataset_tables(bq_dataset(project_id, dataset_id))

# Function to search for the phrase in a table
search_table <- function(table_id) {
  table <- bq_table(project_id, dataset_id, table_id)
  schema <- bq_table_schema(table)
  columns <- schema$fields
  
  results <- lapply(columns, function(column) {
    column_name <- column$name
    column_type <- column$type
    
    # Skip columns that cannot be cast to STRING
    if (column_type %in% c("STRING", "BYTES", "DATE", "DATETIME", "TIMESTAMP", "TIME", "NUMERIC", "BIGNUMERIC", "FLOAT64", "FLOAT", "INTEGER", "INT64")) {
      query <- paste0(
        "SELECT '", table_id, "' AS table_id, '", column_name, "' AS column_name ",
        "FROM `", project_id, ".", dataset_id, ".", table_id, "` ",
        "WHERE CAST(", column_name, " AS STRING) LIKE '%", search_phrase, "%' ",
        "LIMIT 1"
      )
      
      # Print the query being executed
      print(paste("Executing query on table:", table_id, "column:", column_name))
      print(query)
      
      result <- tryCatch({
        bq_project_query(project_id, query)
      }, error = function(e) {
        # Print the error message
        print(paste("Error in table:", table_id, "column:", column_name, ":", e$message))
        NULL
      })
      
      if (!is.null(result)) {
        result_data <- bq_table_download(result)
        if (nrow(result_data) > 0) {
          print(paste("Match found in table:", table_id, "column:", column_name))
          return(data.frame(table_id = table_id, column_name = column_name))
        }
      }
    } else {
      print(paste("Skipping column:", column_name, "of type:", column_type))
    }
    return(NULL)
  })
  
  do.call(rbind, results)
}

# Search all tables and collect results
all_results <- lapply(tables$table_id, search_table)
final_results <- do.call(rbind, all_results)

# Display the results
print(final_results)


NULL


In [ ]:
library(bigrquery)
library(dplyr)

search_data_in_tables <- function(project_id, search_term) {
  # Authenticate and set project
  bq_auth(email = "your-email@example.com")
  bq_project <- bq_project(project_id)
  
  # List all datasets in the project
  datasets <- bq_project_datasets(project_id)
  
  # List to hold datasets and their matching tables
  matching_datasets <- list()
  
  # Loop through each dataset
  for (dataset in datasets) {
    dataset_id <- dataset$dataset
    dataset_ref <- bq_dataset(project_id, dataset_id)
    
    # List all tables in the dataset
    tables <- bq_dataset_tables(dataset_ref)
    
    # List to hold matching tables
    matching_tables <- list()
    
    # Loop through each table
    for (table in tables) {
      table_id <- table$table
      table_ref <- bq_table(dataset_ref, table_id)
      
      # Get the schema of the table to find columns
      schema <- bq_table_fields(table_ref)
      
      # Construct a query to search the term in all columns
      conditions <- list()
      for (field in schema) {
        if (field$type == "STRING") {
          conditions <- append(conditions, sprintf("`%s` LIKE '%%%s%%'", field$name, search_term))
        }
      }
      
      if (length(conditions) > 0) {
        query <- sprintf(
          "SELECT COUNT(1) FROM `%s.%s.%s` WHERE %s",
          project_id, dataset_id, table_id, paste(conditions, collapse = " OR ")
        )
        result <- bq_project_query(project_id, query)
        count <- as.integer(bq_table_download(result)$f0_)
        
        if (count > 0) {
          matching_tables <- append(matching_tables, sprintf("%s.%s.%s", project_id, dataset_id, table_id))
        }
      }
    }
    
    if (length(matching_tables) > 0) {
      matching_datasets[[dataset_id]] <- matching_tables
    }
  }
  
  return(matching_datasets)
}

# Example usage
project_id <- 'yhcr-prd-phm-bia-core'
search_term <- 'asthma'
datasets_with_matching_tables <- search_data_in_tables(project_id, search_term)

for (dataset_id in names(datasets_with_matching_tables)) {
  cat(sprintf("Dataset: %s\n", dataset_id))
  for (table in datasets_with_matching_tables[[dataset_id]]) {
    cat(sprintf("  Table: %s\n", table))
  }
}


In [7]:
library(bigrquery)
library(dplyr)

search_data_in_tables <- function(project_id, search_term) {
  # List all datasets in the project
  datasets <- bq_project_datasets(project_id)
  
  # List to hold tables containing the search term in any column data
  matching_tables <- list()
  
  # Loop through each dataset
  for (dataset in datasets) {
    dataset_id <- dataset$dataset
    dataset_ref <- bq_dataset(project_id, dataset_id)
    
    # List all tables in the dataset
    tables <- tryCatch({
      bq_dataset_tables(dataset_ref)
    }, error = function(e) {
      message(sprintf("Error retrieving tables for dataset %s: %s", dataset_id, e$message))
      return(NULL)
    })
    
    if (is.null(tables)) next
    
    # Loop through each table
    for (table in tables) {
      table_id <- table$table
      table_ref <- tryCatch({
        bq_table(dataset_ref, table_id)
      }, error = function(e) {
        message(sprintf("Error retrieving table %s.%s: %s", dataset_id, table_id, e$message))
        return(NULL)
      })
      
      if (is.null(table_ref)) next
      
      # Get the schema of the table to find columns
      schema <- tryCatch({
        bq_table_fields(table_ref)
      }, error = function(e) {
        message(sprintf("Error retrieving schema for table %s.%s: %s", dataset_id, table_id, e$message))
        return(NULL)
      })
      
      if (is.null(schema)) next
      
      # Construct a query to search the term in all columns
      conditions <- list()
      for (field in schema) {
        if (field$type == "STRING") {
          conditions <- append(conditions, sprintf("`%s` LIKE '%%%s%%'", field$name, search_term))
        }
      }
      
      if (length(conditions) > 0) {
        query <- sprintf(
          "SELECT COUNT(1) FROM `%s.%s.%s` WHERE %s",
          project_id, dataset_id, table_id, paste(conditions, collapse = " OR ")
        )
        result <- tryCatch({
          bq_project_query(project_id, query)
        }, error = function(e) {
          message(sprintf("Error running query for table %s.%s: %s", dataset_id, table_id, e$message))
          return(NULL)
        })
        
        if (is.null(result)) next
        
        count <- tryCatch({
          as.integer(bq_table_download(result)$f0_)
        }, error = function(e) {
          message(sprintf("Error downloading result for table %s.%s: %s", dataset_id, table_id, e$message))
          return(NULL)
        })
        
        if (is.null(count)) next
        
        if (count > 0) {
          matching_tables <- append(matching_tables, sprintf("%s.%s.%s", project_id, dataset_id, table_id))
        }
      }
    }
  }
  
  return(matching_tables)
}

# Example usage
project_id <- 'yhcr-prd-phm-bia-core'
search_term <- 'asthma'
tables <- search_data_in_tables(project_id, search_term)

for (table in tables) {
  cat(sprintf("Table: %s\n", table))
}



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Error retrieving schema for table CB_CDM_V1_50k_Random.Actual Date Of Delivery New: Invalid http request: unsupported HTTP version (or space in URL) [badRequest] 

Error retrieving schema for table CB_MYSPACE_KF.Final steroid codelist: Invalid http request: unsupported HTTP version (or space in URL) [badRequest] 

Error retrieving schema for table CB_MYSPACE_KF.working _table_05_06: Invalid http request: unsupported HTTP version (or space in URL) [badRequest] 

Error running query for table CB_MYSPACE_TL.CommmunityChecks_view_SRPatient_age_IMD: Job yhcr-prd-phm-bia-core.job_Vp_JHCkZF78Wp9gB31O9NA5WolQq.europe-west2 failed

Error running query for table CB_MYSPACE_TL.CommunityChecks_srcode_Y3be2_all: Job yhcr-prd-phm-bia-core.job_R2N-YpEZQ2OVluFJWOk7i3inydEK.europe-west2 failed

Error runn

Table: yhcr-prd-phm-bia-core.CB_2360.Aggregate_Data_MassTest
Table: yhcr-prd-phm-bia-core.CB_2360.COVID_CODE_INTRO
Table: yhcr-prd-phm-bia-core.CB_2360.COVID_SNOMED
Table: yhcr-prd-phm-bia-core.CB_2360.COVID_SNOMED_VALID
Table: yhcr-prd-phm-bia-core.CB_2360.MassTest
Table: yhcr-prd-phm-bia-core.CB_2398.tbl_srcode
Table: yhcr-prd-phm-bia-core.CB_2398_Staging.tbl_srcode
Table: yhcr-prd-phm-bia-core.CB_CDM_Nautilus.concept
Table: yhcr-prd-phm-bia-core.CB_CDM_Nautilus.concept_synonym
Table: yhcr-prd-phm-bia-core.CB_CDM_Nautilus.drug_exposure
Table: yhcr-prd-phm-bia-core.CB_CDM_V1_50k_Random.complete
Table: yhcr-prd-phm-bia-core.CB_CDM_V1_50k_Random.concept
Table: yhcr-prd-phm-bia-core.CB_CDM_V1_50k_Random.concept_synonym
Table: yhcr-prd-phm-bia-core.CB_CDM_V1_50k_Random.condition_concept_lineage
Table: yhcr-prd-phm-bia-core.CB_CDM_V1_50k_Random.drug_exposure
Table: yhcr-prd-phm-bia-core.CB_CDM_VOCAB.concept
Table: yhcr-prd-phm-bia-core.CB_FDM_ASQ.tbl_bdct_srcode
Table: yhcr-prd-phm-bia-cor

In [ ]:
library(bigrquery)
library(future)
library(future.apply)
library(dplyr)

search_data_in_tables <- function(project_id, search_term) {
  # List all datasets in the project
  datasets <- bq_project_datasets(project_id)
  
  # Function to search for the term in a table
  search_table <- function(dataset_id, table_id) {
    dataset_ref <- bq_dataset(project_id, dataset_id)
    table_ref <- bq_table(dataset_ref, table_id)
    
    # Get the schema of the table to find columns
    schema <- tryCatch({
      bq_table_fields(table_ref)
    }, error = function(e) {
      message(sprintf("Error retrieving schema for table %s.%s: %s", dataset_id, table_id, e$message))
      return(NULL)
    })
    
    if (is.null(schema)) return(NULL)
    
    # Construct a query to search the term in all columns
    conditions <- list()
    for (field in schema) {
      if (field$type == "STRING") {
        conditions <- append(conditions, sprintf("`%s` LIKE '%%%s%%'", field$name, search_term))
      }
    }
    
    if (length(conditions) > 0) {
      query <- sprintf(
        "SELECT COUNT(1) as count FROM `%s.%s.%s` WHERE %s",
        project_id, dataset_id, table_id, paste(conditions, collapse = " OR ")
      )
      result <- tryCatch({
        bq_project_query(project_id, query)
      }, error = function(e) {
        message(sprintf("Error running query for table %s.%s: %s", dataset_id, table_id, e$message))
        return(NULL)
      })
      
      if (is.null(result)) return(NULL)
      
      count <- tryCatch({
        as.integer(bq_table_download(result)$count)
      }, error = function(e) {
        message(sprintf("Error downloading result for table %s.%s: %s", dataset_id, table_id, e$message))
        return(NULL)
      })
      
      if (is.null(count)) return(NULL)
      
      if (count > 0) {
        return(sprintf("%s.%s.%s", project_id, dataset_id, table_id))
      }
    }
    
    return(NULL)
  }
  
  # Function to search all tables in a dataset
  search_dataset <- function(dataset) {
    dataset_id <- dataset$dataset
    dataset_ref <- bq_dataset(project_id, dataset_id)
    
    # List all tables in the dataset
    tables <- tryCatch({
      bq_dataset_tables(dataset_ref)
    }, error = function(e) {
      message(sprintf("Error retrieving tables for dataset %s: %s", dataset_id, e$message))
      return(NULL)
    })
    
    if (is.null(tables)) return(NULL)
    
    # Search each table in parallel
    table_results <- future_lapply(tables, function(table) {
      table_id <- table$table
      search_table(dataset_id, table_id)
    }, future.globals = list(search_table = search_table, project_id = project_id, search_term = search_term))
    
    return(unlist(table_results))
  }
  
  # Set up parallel processing
  plan(multisession, workers = availableCores() - 1)
  
  # Search all datasets in parallel
  dataset_results <- future_lapply(datasets, search_dataset, future.globals = list(search_dataset = search_dataset, project_id = project_id, search_term = search_term))
  
  # Combine results
  matching_tables <- unlist(dataset_results)
  return(matching_tables)
}

# Example usage
project_id <- 'yhcr-prd-phm-bia-core'
search_term <- 'asthma'
tables <- search_data_in_tables(project_id, search_term)

for (table in tables) {
  cat(sprintf("Table: %s\n", table))
}
